# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921686050326                   -0.69    5.8         
  2   -7.926159300938       -2.35       -1.22    1.0    436ms
  3   -7.926836715007       -3.17       -2.37    1.8    404ms
  4   -7.926861511959       -4.61       -3.02    3.0    436ms
  5   -7.926861631858       -6.92       -3.34    1.6    410ms
  6   -7.926861666678       -7.46       -3.72    1.8    332ms
  7   -7.926861680653       -7.85       -4.41    1.1    313ms
  8   -7.926861681819       -8.93       -4.98    2.2    377ms
  9   -7.926861681862      -10.37       -5.28    1.6    380ms
 10   -7.926861681872      -11.00       -6.02    1.8    330ms
 11   -7.926861681873      -12.10       -6.83    2.2    376ms
 12   -7.926861681873      -13.62       -7.30    2.8    403ms
 13   -7.926861681873      -14.57       -8.12    2.1    369ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921683726770                   -0.69    5.8         
  2   -7.926164425905       -2.35       -1.22    1.0    440ms
  3   -7.926837789553       -3.17       -2.37    1.6    354ms
  4   -7.926861519227       -4.62       -3.05    2.9    390ms
  5   -7.926861650220       -6.88       -3.44    1.8    338ms
  6   -7.926861672081       -7.66       -3.86    1.8    409ms
  7   -7.926861680321       -8.08       -4.30    1.4    314ms
  8   -7.926861681742       -8.85       -4.83    1.8    335ms
  9   -7.926861681862       -9.92       -5.30    1.9    342ms
 10   -7.926861681871      -11.01       -5.91    1.9    415ms
 11   -7.926861681873      -11.91       -6.70    2.0    371ms
 12   -7.926861681873      -13.29       -7.80    2.6    404ms
 13   -7.926861681873   +  -15.05       -8.52    3.9    488ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921628692247                   -0.69    5.9         
  2   -7.926166675228       -2.34       -1.22    1.0    310ms
  3   -7.926841068469       -3.17       -2.37    1.8    369ms
  4   -7.926864942940       -4.62       -3.02    2.5    389ms
  5   -7.926865047822       -6.98       -3.36    2.0    394ms
  6   -7.926865077557       -7.53       -3.73    1.8    317ms
  7   -7.926865091620       -7.85       -4.37    1.0    289ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
